In [34]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

In [35]:
# Configuration constants
WEAVIATE_URL = "https://v3dtdzg0skwvxinygzckra.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = "Wn0Zd8LOYfOwAyPBgxtnsoIpgLGEOTM7iHb0"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [36]:
# Setup vector store
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [37]:
SYSTEM_PROMPT = PromptTemplate(
    template=(
                "Dựa trên tài liệu đã trích xuất, hãy phân tích và trả lời câu hỏi.\n"
                "Câu hỏi: {question}\n"
                "Tài liệu: {context}\n"
                "Hãy suy nghĩ từng bước:\n"
                "1. Phân tích xem thông tin có đủ và liên quan không?\n"
                "2. Nếu chưa đủ, hãy đưa ra truy vấn mới để tìm thêm thông tin\n"
                "3. Nếu đã đủ, đưa ra câu trả lời cuối cùng\n\n"
                "Hãy trả lời theo định dạng sau:\n"
                "Phân tích: <phân tích thông tin hiện có>\n"
                "Quyết định: [Cần thêm thông tin/Đã đủ thông tin]\n"
                "Truy vấn tiếp theo: <truy vấn mới> (nếu cần)\n"
                "Câu trả lời cuối cùng: <câu trả lời> (nếu đã đủ thông tin)\n"
            )
)

In [38]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True),
)

In [39]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=5,
    alpha=0.5  # Balance between keyword and semantic search
)

In [40]:
from llama_index.llms.openai import OpenAI

In [41]:
llm = OpenAI(model="gpt-4o-mini", api_key="")

In [42]:
# Test reasoning and retrieval
async def test_autonomous_reasoning(question):
    retrieved_docs = retriever.retrieve(question)
    docs_text = "\n".join([doc.text for doc in retrieved_docs])
    prompt = SYSTEM_PROMPT.format(question=question, documents=docs_text)
    response = await llm.acomplete(prompt)
    return response.text.strip()


In [43]:
import asyncio

question = "Không đội mũ bảo hiểm thì bị phạt bao nhiêu tiền?"

# Use `await` directly in Jupyter Notebook
answer = await test_autonomous_reasoning(question)

print("AI Answer:", answer)


AI Answer: Phân tích: Tài liệu hiện tại không cung cấp thông tin cụ thể về mức phạt tiền khi không đội mũ bảo hiểm. Do đó, tôi không thể xác định được số tiền phạt cụ thể cho hành vi này.

Quyết định: Cần thêm thông tin

Truy vấn tiếp theo: "Mức phạt tiền khi không đội mũ bảo hiểm là bao nhiêu theo quy định hiện hành?"

Câu trả lời cuối cùng: Chưa có câu trả lời do thiếu thông tin.


In [45]:
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print("="*50)

Result 1:

Text: 4 . phạt tiền từ 400.000 đồng đến 600.000 đồng đối_với người điều_khiển xe thực_hiện một trong các hành_vi vi_phạm sau đây : a ) gây tai_nạn giao_thông không dừng ngay phương_tiện , không giữ nguyên hiện_trường , không trợ_giúp người bị nạn , không ở lại hiện_trường hoặc không đến trình_báo ngay với cơ_quan công_an , ủy ban nhân_dân nơi gần nhất ; b ) điều_khiển xe trên đường mà trong máu hoặc hơi thở có nồng_độ cồn vượt quá 80 miligam / 100 mililít máu hoặc vượt quá 0,4 miligam / 1 lít khí thở ; c ) không chấp_hành yêu_cầu kiểm_tra về nồng_độ cồn của người thi_hành công_vụ ; d ) người điều_khiển xe_đạp máy không đội “ mũ_bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ_bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách khi tham_gia giao_thông trên đường_bộ ; đ ) chở người ngồi trên xe_đạp máy không đội “ mũ_bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ_bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở 